In [11]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.v2 as transforms

from binaryconnect import BC
from models import DenseNet121
from utils import get_device

In [19]:
device = get_device()
model = DenseNet121()
model = model.to(device)

In [13]:
print_size_of_model(model)

Size (MB): 28.353676


In [ ]:
quantized_model = quantise_dynamic(model)

In [24]:
print_size_of_model(quantized_model)

Size (MB): 28.323778


In [35]:
model = model.half()

In [56]:
for m in quantized_model.modules():
    print(type(m))
    if isinstance(m, nn.Linear):
        break

<class 'models.densenet.DenseNet'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.container.Sequential'>
<class 'models.densenet.Bottleneck'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'models.densenet.Bottleneck'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'models.densenet.Bottleneck'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'models.densenet.Bottleneck'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'models.densenet.Bo

In [ ]:
supported_backends = ["fbgemm", "x86", "qnnpack", "onednn"]

In [25]:
# Download CIFAR-10 dataset (train and test)
train_dataset = subset_data(CIFAR10(root=ROOT_DIR, train=True, download=True, transform=DEFAULT_TRANSFORM), 5000)
test_dataset = CIFAR10(root=ROOT_DIR, train=False, download=True, transform=DEFAULT_TRANSFORM)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
from torch import dtype as T_dtype
from torch.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.utils.data.dataloader import DataLoader

QCONFIG = get_default_qconfig('fbgemm')
QCONFIG_DEFAULT = {
    "": QCONFIG,
    "object_type": [
        (torch.nn.Conv2d, QCONFIG),
        (torch.nn.Linear, QCONFIG),
        (torch.nn.ReLU, QCONFIG),
        (torch.nn.BatchNorm2d, QCONFIG),
    ]
}


def quantise_dynamic(
        model: nn.Module,
        spec: tuple = {torch.nn.Linear},
        dtype: T_dtype = torch.qint8,
    ) -> nn.Module:
    return torch.quantization.quantize_dynamic(
        model, spec, dtype=dtype,
    )

def quantise_static(
        model: nn.Module,
        calibration_loader: DataLoader,
        qconfig_dict: dict = QCONFIG_DEFAULT,
    ) -> nn.Module:
    model = model.to('cpu')
    example_inputs = (next(iter(calibration_loader))[0],)
    prepared_model = prepare_fx(model, qconfig_dict, example_inputs)
    return convert_fx(prepared_model)
    

In [ ]:
import torch
from torch.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx

model = model.to('cpu')
model.eval()



example_inputs = (next(iter(train_loader))[0],)
prepared_model = prepare_fx(model, qconfig_dict, example_inputs)
quantized_model = convert_fx(prepared_model)

print(quantized_model)


/users/local/b24richa/EDL/.venv/lib/python3.12/site-packages/torch/ao/quantization/quantize_fx.py:146: FutureWarning: Passing a QConfig dictionary to prepare is deprecated and will not be supported in a future version. Please pass in a QConfigMapping instead.
  prepared = prepare(
/users/local/b24richa/EDL/.venv/lib/python3.12/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/users/local/b24richa/EDL/.venv/lib/python3.12/site-packages/torch/ao/quantization/observer.py:1318: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


GraphModule(
  (conv1): QuantizedConv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
  (dense1): Module(
    (0): Module(
      (bn1): QuantizedBNReLU2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): QuantizedConvReLU2d(64, 128, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (conv2): QuantizedConv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
    )
    (1): Module(
      (bn1): QuantizedBNReLU2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): QuantizedConvReLU2d(96, 128, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (conv2): QuantizedConv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
    )
    (2): Module(
      (bn1): QuantizedBNReLU2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Quant

In [29]:
print_size_of_model(quantized_model)

Size (MB): 7.89627


In [20]:
model = model.to("cpu")
for m in model.modules():
    if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
        m.qconfig = torch.quantization.get_default_qconfig('fbgemm')
        torch.quantization.prepare(m, inplace=True)
        torch.quantization.convert(m, inplace=True)
    else:
        continue

/users/local/b24richa/EDL/.venv/lib/python3.12/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [21]:
print(model)

DenseNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (dense1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [16]:
model = model.to("cuda")

In [17]:
print_size_of_model(model)

Size (MB): 28.353676


In [9]:
acc = test(test_loader, model, criterion, "cuda", half=True)

In [9]:

mymodelbc = BC(model)
optimiser = optim.SGD(model.parameters())
criterion = nn.CrossEntropyLoss()
device = "cuda"

train_loader = get_train_cifar10(transform_test, subset=True)

In [10]:

best_acc = 0
start_epoch = 0
n_epochs = 10

for epoch in range(start_epoch, start_epoch+n_epochs):
    mymodelbc.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimiser.zero_grad()
        outputs = mymodelbc(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimiser.step()
        mymodelbc.clip()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()



KeyboardInterrupt: 

In [38]:
from utils import train


def quantise_aware_train(
        train_loader: DataLoader,
        model: nn.Module,
        optimiser: "Optimizer",
        criterion,
        qconfig: str = "fbgemm",
        device: str = "cuda",
        n_epochs: int = 500,
    ) -> None:
    model.qconfig = torch.quantization.get_default_qat_qconfig(qconfig)
    torch.quantization.prepare_qat(model, inplace=True)
    
    for _ in range(n_epochs):
        acc, loss = train(
            train_loader,
            model,
            optimiser,
            criterion,
            device,
        )

    model = model.to('cpu')
    torch.quantization.convert(model, inplace=True)
    model = model.to(device)

In [39]:
from torchvision.transforms import v2 as transformsv2

In [4]:
from utils import get_cifar10_train_val_loaders, ROOT_DIR, DEFAULT_TRANSFORM, print_size_of_model, subset_data

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader

train_loader, val_loader = get_cifar10_train_val_loaders(subset=True, num_subset=5000)


# Download CIFAR-10 dataset (train and test)
train_dataset = subset_data(CIFAR10(root=ROOT_DIR, train=True, download=True, transform=DEFAULT_TRANSFORM), 5000)
test_dataset = CIFAR10(root=ROOT_DIR, train=False, download=True, transform=DEFAULT_TRANSFORM)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


# Define a simple model (e.g., for QAT)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.fc1 = nn.Linear(16 * 30 * 30, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = x.flatten(1)
        x = self.fc1(x)
        return x

# Initialize model, move to CUDA
model = SimpleModel().to('cuda')

print_size_of_model(model)

# QAT preparation
model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(model, inplace=True)

# Set up optimizer and criterion
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(10):  # Example epochs
    print(epoch)
    model.train()
    for inputs, targets in train_loader:  # Assuming a DataLoader `train_loader`
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Convert to quantized model after training
torch.quantization.convert(model, inplace=True)

# Now, the model is quantized and can be used for inference.
print_size_of_model(model)

Size (MB): 0.579704
0


/users/local/b24richa/EDL/.venv/lib/python3.12/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


1
2
3
4
5
6
7
8
9


RuntimeError: Unsupported qscheme: per_channel_affine

In [7]:
model = model.to('cpu')  # Move the model back to CPU for conversion
torch.quantization.convert(model, inplace=True)

SimpleModel(
  (conv1): QuantizedConv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.06295403838157654, zero_point=63)
  (fc1): QuantizedLinear(in_features=14400, out_features=10, scale=0.1440950483083725, zero_point=59, qscheme=torch.per_channel_affine)
)

In [8]:
print_size_of_model(model)

Size (MB): 0.148866


In [9]:
model = model.to('cuda')

In [10]:
print_size_of_model(model)

Size (MB): 0.148866
